<a href="https://colab.research.google.com/github/healthonrails/annolid/blob/main/docs/tutorials/yolov8_tracking_tutorial.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# Annolid on YOLOv8 
This notebook show examples for how to upload a custom dataset, train a new model based on the dataset, and inference on provided videos.


# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [ ]:
!git clone --recurse-submodules https://github.com/mikel-brostrom/yolov8_tracking.git  # clone repo
%cd /content/yolov8_tracking
%pip install ultralytics
%pip install -qr requirements.txt  # install dependencies
%pip install thop                  # install dependencies
%pip install sentry_sdk
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Upload and train on your custom dataset

In [ ]:
from google.colab import files
upload = files.upload()
dataset =  list(upload.keys())[0]

In [ ]:
#!unzip /content/datasets/test_mouse.zip -d /content/datasets/
!unzip $dataset -d /content/datasets

## Find the data.yaml file and change the train, val, and test absolute paths

In [ ]:
# find data.yaml file
import glob
data_ymals = glob.glob('/content/datasets/*/data.yaml')
print("The data.yaml file is located at: ", data_ymals)
dataset_dir = os.path.dirname(data_ymals[0])
print("The dataset directory is located at :", dataset_dir)

This function first searches for a data.yaml file in the specified root directory and its subdirectories. If the file is found, it reads its contents and replaces any lines that match the old_line argument with the new_line argument. It then saves the modified file and prints a message confirming the replacement. If the file is not found, it raises a ValueError.

In [ ]:
import os
def replace_yaml_lines(root_dir, old_line, new_line):
    # Find data.yaml file
    for root, dirs, files in os.walk(root_dir):
        if 'data.yaml' in files:
            yaml_path = os.path.join(root, 'data.yaml')
            break
    else:
        raise ValueError('No data.yaml file found in root directory or subdirectories')
    
    # Replace specified lines
    with open(yaml_path, 'r') as f:
        lines = f.readlines()
    
    with open(yaml_path, 'w') as f:
        for line in lines:
            if old_line in line:
                line = new_line + '\n'
            f.write(line)
    
    print(f'Replaced {old_line} with {new_line} in {yaml_path}')


In [ ]:
replace_yaml_lines(dataset_dir, 'train: ../train/images', f'train: {dataset_dir}/train/images')
replace_yaml_lines(dataset_dir, 'val: ../val/images', f'val: {dataset_dir}/val/images')
replace_yaml_lines(dataset_dir, 'test: ../test/images', f'test: {dataset_dir}/val/images')

## Train a new model based on the custom new dataset


In [ ]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n-seg.yaml")  # build a new model from scratch
# load a pretrained model (recommended for training)
model_name = "yolov8n-seg.pt" #@param ['yolov8n-seg.pt','yolov8x-seg.pt','yolov8n.pt','yolov8x.pt']
model = YOLO(model_name) 
data_yaml_file = data_ymals[0] 
num_epoches = 100 #@param
# Train the model
model.train(data=data_yaml_file, epochs=num_epoches)

# Download data

Get test video from repo and extract the first 2 seconds of it or your can upload your video by click the file upload button under Files menu on the left

In [ ]:

# upload your video or get the test video from the repo
!wget -nc https://github.com/mikel-brostrom/yolov8_tracking/releases/download/v.2.0/test.avi
# extract 3 seconds worth of video frames of it
!yes | ffmpeg -ss 00:00:00 -i test.avi -t 00:00:01 -vf fps=30 out.avi

## Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935. 

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time

In [ ]:
video_file = '/content/directedsong_20221007_clip.mp4' #@param
!python track.py --yolo-weights yolov8m-seg.pt --tracking-method ocsort --reid-weights osnet_x0_25_msmt17.pt --device 0 --source $video_file --save-vid --save-crop --save-txt

# Download and predict a video from YouTube

In [ ]:
!pip install pytube

In [ ]:
from pytube import YouTube
import os

def downloadYouTube(videourl, path):

    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    if not os.path.exists(path):
        os.makedirs(path)
    yt.download(path)

downloadYouTube('https://youtu.be/wepTdonuH2M', '/content')

In [ ]:
video_file = '/content/zebra_finch.mp4' #@param
!python track.py --yolo-weights yolov8x-seg.pt --tracking-method ocsort --reid-weights osnet_x0_25_msmt17.pt --device 0 --source $video_file --save-vid --save-txt


In [ ]:
import pandas as pd
##https://github.com/mikel-brostrom/yolov8_tracking/blob/master/track.py#L243
## Write MOT compliant results to file
##                            with open(txt_path + '.txt', 'a') as f:
##                                f.write(('%g ' * 10 + '\n') % (frame_idx + 1, id, bbox_left,  # MOT format
##                                                               bbox_top, bbox_w, bbox_h, -1, -1, -1, i))
# define the column names
columns = ['Frame_ID', 'Object_ID', 'bbox_left', 'bbox_top', 'Width', 'Height', 'Confidence', 'Class_ID','Visibility_ratio','idx']
df = pd.read_csv('/content/yolov8_tracking/runs/track/exp/tracks/zebra_finch.txt',header=None,delimiter='\s')

In [ ]:
df.columns = columns

In [ ]:
df.head()

In [ ]:
class_name = 'bird'

In [ ]:
annolid_columns = ['frame_number','x1','y1','x2','y2','instance_name','class_score','segmentation','tracking_id']

In [ ]:
df['x1'] = df.bbox_left
df['y1'] = df.bbox_top
df['x2'] = df.bbox_left + df.Width
df['y2'] = df.bbox_top + df.Height
df['frame_number'] = df.Frame_ID
df['instance_name'] = f"{class_name}"
df['class_score'] = 1
df['segmentation'] = 'None'
df['tracking_id'] = df.Object_ID



In [ ]:
df_annolid = df[annolid_columns]

In [ ]:
df_annolid.head()

In [ ]:
tracking_csv_file = f'{video_file.split(".")[0]}_tracking_results.csv'
df_annolid.to_csv(tracking_csv_file)

In [ ]:
from google.colab.files import download

In [ ]:
download(tracking_csv_file)

# Show results

https://stackoverflow.com/questions/60977179/how-to-play-avi-file-in-google-colab

https://stackoverflow.com/questions/57377185/how-play-mp4-video-in-google-colab

Compress the video file to a format supported by Google Colab (mpeg4 (native) -> h264 (libx264))

In [ ]:
!ffmpeg -i /content/yolov8_tracking/runs/track/exp11/bird.mp4 -vf fps=30 -vcodec libx264 output.mp4

Get the file content into data_url

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

Display it with HTML

In [ ]:
HTML("""
<video controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Reference: https://github.com/mikel-brostrom/yolov8_tracking